In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recognizing-faces-in-the-wild/train_relationships.csv
/kaggle/input/recognizing-faces-in-the-wild/test-private-labels.zip
/kaggle/input/recognizing-faces-in-the-wild/sample_submission.csv
/kaggle/input/recognizing-faces-in-the-wild/train.zip
/kaggle/input/recognizing-faces-in-the-wild/README.md
/kaggle/input/recognizing-faces-in-the-wild/test-public-lists.zip
/kaggle/input/recognizing-faces-in-the-wild/train-faces.zip
/kaggle/input/recognizing-faces-in-the-wild/test.zip
/kaggle/input/recognizing-faces-in-the-wild/test-private-lists.zip
/kaggle/input/recognizing-faces-in-the-wild/test-private-faces.zip
/kaggle/input/recognizing-faces-in-the-wild/test-public-faces.zip
/kaggle/input/recognizing-faces-in-the-wild/test-faces.zip


In [2]:
print(os.listdir("../input/recognizing-faces-in-the-wild"))
# train_zip_file = "../input/recognizing-faces-in-the-wild/train.zip"
# extract_train = "../input/recognizing-faces-in-the-wild/train/"
! unzip "../input/recognizing-faces-in-the-wild/train.zip" -d "../working/train/"

['train_relationships.csv', 'test-private-labels.zip', 'sample_submission.csv', 'train.zip', 'README.md', 'test-public-lists.zip', 'train-faces.zip', 'test.zip', 'test-private-lists.zip', 'test-private-faces.zip', 'test-public-faces.zip', 'test-faces.zip']
Archive:  ../input/recognizing-faces-in-the-wild/train.zip
   creating: ../working/train/F0002/
   creating: ../working/train/F0005/
   creating: ../working/train/F0009/
   creating: ../working/train/F0010/
   creating: ../working/train/F0016/
   creating: ../working/train/F0017/
   creating: ../working/train/F0020/
   creating: ../working/train/F0022/
   creating: ../working/train/F0023/
   creating: ../working/train/F0024/
   creating: ../working/train/F0030/
   creating: ../working/train/F0031/
   creating: ../working/train/F0036/
   creating: ../working/train/F0038/
   creating: ../working/train/F0039/
   creating: ../working/train/F0040/
   creating: ../working/train/F0041/
   creating: ../working/train/F0044/
   creating: ../wo

In [3]:
! cp "../input/recognizing-faces-in-the-wild/train_relationships.csv" "../working/"

In [4]:
print(os.listdir("../working"))

['__notebook__.ipynb', 'train', 'train_relationships.csv']


In [5]:
!pip install facenet-pytorch==2.5.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.0 MB/s eta 0:00:00


In [6]:
import pandas as pd
from glob import glob
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.utils
import torchvision.datasets as dset

from torch import optim
from torch.utils.data import DataLoader,Dataset
from torchvision.models import *
# from torchsummary import summary
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import numpy as np
from PIL import Image
import time
import random

from facenet_pytorch import MTCNN,InceptionResnetV1
import torchvision.models as models
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [7]:
relationshipsCSV = "../working/train_relationships.csv"
train_images_folder = "../working/train/"

In [8]:
available_images = glob(train_images_folder + "*/*/*.jpg")
print("Total Length: ", len(available_images))
for a in range(0, len(available_images)):
    available_images[a] = available_images[a].replace("\\", "/")
all_ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in available_images] #all the people
print(all_ppl[:5])

Total Length:  12379
['F0099/MID8', 'F0099/MID8', 'F0099/MID8', 'F0099/MID8', 'F0099/MID9']


In [9]:
personPathFile = defaultdict(list)
for x in available_images:
    personPathFile[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

relationships = pd.read_csv(relationshipsCSV)
relationship_pairs = [(row['p1'], row['p2']) for index, row in relationships.iterrows()] # Create a list of tuples
relationship_pairs = [x for x in relationship_pairs if x[0] in all_ppl and x[1] in all_ppl] #data cleaning

relationshipDict = defaultdict(list)
for item in relationship_pairs:
  relationshipDict[item[0]].append(item[1]) #you do not need to consider the item[1] because the CSV is already pretty clean

#create more data by finding the 'negative pair'
notRelationshipDict = defaultdict(list)
set_all_ppl = set(all_ppl)
#for each key(person) in the relationship dictionary, find the non-relations
count = 0
for k,v in relationshipDict.items():
    notRelationshipDict[k]=list(set_all_ppl-set(v))

trainData = []

targetRelatedCount = 36000 #32k data
relatedCount = 0
while relatedCount<targetRelatedCount:
  for k,v in relationshipDict.items():
     for relation in v:
      i2 = random.randint(0, len(personPathFile[k])-1) #random photo of person1
      i3 = random.randint(0, len(personPathFile[relation])-1) #random photo of person2
      trainData.append((personPathFile[k][i2], personPathFile[relation][i3], 1))
      relatedCount+=1
      if relatedCount>=targetRelatedCount:
          break
     if relatedCount>=targetRelatedCount:
        break


trainData = set(trainData)
trainData = list(trainData)

positiveRelationsCount = len(trainData)
print("Current Length of positive relationships: ", len(trainData))

notRelationAddedCount = 0

#might choose the same relation but handled later on when convert trainData to set and back to list
while notRelationAddedCount<positiveRelationsCount:
  for k,v in notRelationshipDict.items():
     i1 = random.randint(0, len(v)-1)
     i2 = random.randint(0, len(personPathFile[k])-1) #random photo of person1
     i3 = random.randint(0, len(personPathFile[v[i1]])-1) #random photo of person2
     trainData.append((personPathFile[k][i2], personPathFile[v[i1]][i3], 0))
     notRelationAddedCount+=1
     if notRelationAddedCount>=positiveRelationsCount:
      break

print("Current Length of not relationships: ", notRelationAddedCount)
print("Current Length of total relationships: ", len(trainData))

trainData = set(trainData)
trainData = list(trainData)
print("Current Length of total relationships: ", len(trainData))

Current Length of positive relationships:  23891
Current Length of not relationships:  23891
Current Length of total relationships:  47782
Current Length of total relationships:  47776


In [10]:
class trainingDataset(Dataset):
    def __init__(self, _data, _transform=None):
        self.data = _data #choose either train or val dataset to use
        self.transform = _transform


    def __getitem__(self,index):

        first_img_path = self.data[index][0]
        second_img_path = self.data[index][1]
        img1Opened = Image.open(first_img_path)
        # img1Opened_resized = img1Opened.resize(newsize)
        img2Opened = Image.open(second_img_path)
        # img2Opened_resized = img2Opened.resize(newsize)


        #it is important to transform the images into tensor format
        if self.transform is not None:
            img1Opened = self.transform(img1Opened)
            img2Opened = self.transform(img2Opened)
        return img1Opened, img2Opened , self.data[index][2]

    def __len__(self):
        return len(self.data)

In [11]:
BATCH_SIZE=64
IMG_SIZE=100
NUM_WORKERS = 4

trainSet, valSet = train_test_split(trainData, test_size=0.1, random_state=42)
#Training set and training loader
def createTrain(transformations):
  trainset = trainingDataset(_data=trainSet, _transform=transforms.Compose(transformations))

  trainloader = DataLoader(trainset, shuffle=True, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE)

  return trainloader



valset = trainingDataset(_data=valSet, _transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)), transforms.ToTensor()]))


valloader = DataLoader(valset, shuffle=True, num_workers=NUM_WORKERS, batch_size=BATCH_SIZE)

In [12]:
class SiameseNetwork(nn.Module):
    def __init__(self,mdl):

        super(SiameseNetwork, self).__init__()

        model = mdl

        for param in model.parameters():
          param.require_grad = False

        self.cnn1 = torch.nn.Sequential(*(list(model.children())[:-1][:-3]))


        model.classifier = nn.Sequential(
            nn.Linear(3584,2048),
            nn.ReLU(),
            nn.Dropout(0.55),  # Add dropout for regularization
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256), # Apply batch normalization
            nn.Linear(256, 2)
        )


        self.fc = model.classifier


    def forward_one(self, x):
        return self.model(x)

    def forward(self, input1, input2):
        output1 = self.cnn1(input1)
        output1 = output1.view(output1.size()[0], -1)
        output2 = self.cnn1(input2)
        output2 = output2.view(output2.size()[0], -1)


        output = torch.cat((output1, output2),1)
        output = self.fc(output)
        return output

In [13]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim=True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [14]:
# Create the Siamese network
net = SiameseNetwork(InceptionResnetV1(pretrained='vggface2', classify=False)).cuda()

# Define the contrastive loss
criterion = nn.CrossEntropyLoss()

# Define the optimizer (e.g., Adam)
optimizer = optim.Adam(net.parameters(), lr=0.0005)

# model_on_gpu = model.to(device)

print(len(valset)) #checking the length of the validation set
print(len(trainSet))

  0%|          | 0.00/107M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/facenet_pytorch/models/inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)

4778
42998


In [15]:
BATCH_SIZE=64
NUMBER_EPOCHS=30
epoch_accuracies = np.array([])

net.train()
overall_start = time.time()
for epoch in range(1,NUMBER_EPOCHS+1):
    epoch_start = time.time()
    print("--------------------------------------------------------------------------------------------------------------", )
    total_loss = 0

    #Change Data Augmentation every other epoch
    if epoch % 10 == 0 or epoch %10 == 1 or epoch %10 == 2 or epoch %10 == 3:
      print("Data Augmentation: None")
      trainloader = createTrain([transforms.Resize((IMG_SIZE,IMG_SIZE)),transforms.ToTensor()])
    elif epoch %10 == 4 or epoch %10 == 5:
       print("Data Augmentation: RandomGrayScale(0.5)")
       trainloader = createTrain([transforms.Resize((IMG_SIZE,IMG_SIZE)),transforms.RandomGrayscale(p=0.5),transforms.ToTensor()])
    elif epoch %10 == 6 or epoch %10 == 7:
      print("Data Augmentation: RandomCrop((90,90)),RandomGrayScale(0.8), RandomHorizontalFlip, GaussianBlur(kernel_size = 5, sigma=(0.1, 3.0)")
      trainloader = createTrain([transforms.RandomCrop((80,80)),transforms.Resize((IMG_SIZE,IMG_SIZE)),transforms.RandomGrayscale(p=0.8),transforms.RandomHorizontalFlip(),transforms.GaussianBlur(kernel_size = 5, sigma=(0.1, 3.0)),transforms.ToTensor()])
    elif epoch %10 == 8 or epoch %10 == 9:
      print("Data Augmentation: RandomGrayScale(0.8), RandomHorizontalFlip, ColorJitter(brightness=0.7, contrast=0.3),")
      trainloader = createTrain([transforms.Resize((IMG_SIZE,IMG_SIZE)),transforms.RandomGrayscale(p=0.5),transforms.RandomHorizontalFlip(),transforms.ColorJitter(brightness=0.7, contrast=0.3),transforms.ToTensor()])
    else:
      print("Data Augmentation: None")
      trainloader = createTrain([transforms.Resize((IMG_SIZE,IMG_SIZE)),transforms.ToTensor()])

    for i, data in enumerate(trainloader,0):
        img0, img1 , labels = data
        img0, img1 , labels = img0.cuda(), img1.cuda() , labels.cuda()
        optimizer.zero_grad()
        outputs = net(img0,img1)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    epoch_end = time.time()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data in valloader:
            img0, img1 , labels = data
            img0, img1 , labels = img0.cuda(), img1.cuda() , labels.cuda()
            outputs = net(img0,img1)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    accuracy = 100 * correct_val / total_val
    epoch_accuracies = np.append(epoch_accuracies, accuracy)
    print(f"Epoch {epoch}/{NUMBER_EPOCHS}, Loss: {total_loss/len(trainloader)}, Accuracy: {accuracy}, Total Time: {(epoch_end-epoch_start)/60} min")


overall_end = time.time()
best_accuracy = np.amax(epoch_accuracies)
best_epoch = epoch_accuracies.argmax()
print('END TRAINING: BEST EPOCH: ', best_epoch+1, ' WITH ACCURACY: ', best_accuracy, ' TOTAL TIME: ', (overall_end-overall_start)/60, 'min')

--------------------------------------------------------------------------------------------------------------
Data Augmentation: None
Epoch 1/30, Loss: 0.6308611716099438, Accuracy: 67.2247802427794, Total Time: 2.1145713686943055 min
--------------------------------------------------------------------------------------------------------------
Data Augmentation: None
Epoch 2/30, Loss: 0.5720641153997609, Accuracy: 69.96651318543323, Total Time: 2.0955394585927327 min
--------------------------------------------------------------------------------------------------------------
Data Augmentation: None
Epoch 3/30, Loss: 0.5212174167501784, Accuracy: 73.81749686061113, Total Time: 2.09659130970637 min
--------------------------------------------------------------------------------------------------------------
Data Augmentation: RandomGrayScale(0.5)
Epoch 4/30, Loss: 0.5122089000596177, Accuracy: 74.84303055671829, Total Time: 2.0858206590016684 min
---------------------------------------

In [16]:
!unzip "../input/recognizing-faces-in-the-wild/test.zip" -d "../working/test/"

Archive:  ../input/recognizing-faces-in-the-wild/test.zip
  inflating: ../working/test/face00111.jpg  
  inflating: ../working/test/face04253.jpg  
  inflating: ../working/test/face05610.jpg  
  inflating: ../working/test/face05478.jpg  
  inflating: ../working/test/face05298.jpg  
  inflating: ../working/test/face02061.jpg  
  inflating: ../working/test/face03115.jpg  
  inflating: ../working/test/face04708.jpg  
  inflating: ../working/test/face00826.jpg  
  inflating: ../working/test/face03021.jpg  
  inflating: ../working/test/face04999.jpg  
  inflating: ../working/test/face00696.jpg  
  inflating: ../working/test/face01787.jpg  
  inflating: ../working/test/face00552.jpg  
  inflating: ../working/test/face04364.jpg  
  inflating: ../working/test/face03354.jpg  
  inflating: ../working/test/face01039.jpg  
  inflating: ../working/test/face01392.jpg  
  inflating: ../working/test/face01252.jpg  
  inflating: ../working/test/face04815.jpg  
  inflating: ../working/test/face03253.jpg

In [17]:
print(os.listdir("../working/"))
! cp "../input/recognizing-faces-in-the-wild/sample_submission.csv" "../working/"

['__notebook__.ipynb', 'test', 'train', 'train_relationships.csv']


In [18]:
class testDataset(Dataset):

    def __init__(self,transform=None):
        self.test_df = pd.read_csv('../working/sample_submission.csv')
        self.transform = transform

    def __getitem__(self,index):

        img0_path = self.test_df.iloc[index].img_pair.split("-")[0]
        img1_path = self.test_df.iloc[index].img_pair.split("-")[1]

        img0 = Image.open('../working/test/'+img0_path)
        img1 = Image.open('../working/test/'+img1_path)

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)

        return img0, img1

    def __len__(self):
        return len(self.test_df)


testset = testDataset(transform=transforms.Compose([transforms.Resize((IMG_SIZE,IMG_SIZE)),
                                                                      transforms.ToTensor()
                                                                      ]))
testloader = DataLoader(testset,
                        shuffle=False,
                        num_workers=0,
                        batch_size=1)

net.eval()
test_df = pd.read_csv('../working/sample_submission.csv')
predictions=[]
with torch.no_grad():
    for data in testloader:
        img0, img1 = data
        img0, img1 = img0.cuda(), img1.cuda()
        output = net(img0,img1)
        probabilities = torch.softmax(output, dim=1)
        
        class_1_probabilities = probabilities[:, 1]
        predictions = np.concatenate((predictions,class_1_probabilities.cpu().numpy()),0)

test_df['is_related'] = predictions
# test_df.to_csv("../output/test_Results.csv", index=False)
test_df.to_csv("../working/test_Results.csv", index=False)